In [228]:
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go

# First calculate anti-discrimination

In [229]:
df = pd.read_csv('all_data.csv')
df_n = pd.DataFrame()
df_n['state'] = df['state']
df_n['abbr'] = df['abbr']

# opportunity
df_n['housing'] = (df['housing_so'] + df['housing_gi'])/2
df_n['employment'] = (df['employment_public_so'] + df['employment_public_gi'] +
    df['employment_private_so'] + df['employment_private_gi'])/4
df_n['opportunity'] = 1 - (df_n['housing'] + df_n['employment'])/2
df_n['opportunity'] = df_n['opportunity'] != 0
df_n = df_n.drop(['housing', 'employment'], axis=1)

# control
df_n['control'] = (df['dems_perc_senate'] + df['dems_perc_house'] + df['dem_gov'])/100/3
df_n['all_control'] = (df['dems_perc_senate'] > 50) & (df['dems_perc_house'] > 50) & df['dem_gov']

# support
df_n['support'] = df['poll_favor']/100

In [230]:
def norm(c):
    return (c-c.min())/(c.max()-c.min())

df_f = df_n[df_n['opportunity']].copy()
df_f['opp_index'] = round((norm(df_f['support']) + norm(df_f['control']))/2, 2)*100
df_f.head()

,state,abbr,opportunity,control,all_control,support,opp_index
0,Alabama,AL,True,0.165100,False,0.59,35.0
1,Alaska,AK,True,0.241667,False,0.68,75.0
2,Arizona,AZ,True,0.305533,False,0.68,85.0
3,Arkansas,AR,True,0.165700,False,0.56,25.0
8,Florida,FL,True,0.272233,False,0.68,79.0


## Create the HTML text for the graph

In [231]:
# an unhealthy, ugly way to make the appropriate text html
def adder(a, b, row):
    so = "s.o" if not row[a] else ""
    gi = "g.i." if not row[b] else ""
    if so != "" and gi != "":
        return so + ", " + gi
    else:
        return so + gi

def make_text(row):
    # normal stuff
    text = row['state']
    s = str(round(row['poll_favor'], 0))
    c = round((row['dems_perc_senate'] + row['dems_perc_house'] + row['dem_gov'])/3, 0)
    c = str(c)
    text += "<br>Support: {}<br>Dem control: {}".format(s, c)
    
    # stuff that changes
    text += "<br><br>LACKS"
    if not (row['housing_so'] and row['housing_gi']):
        a = adder('housing_so', 'housing_gi', row)
        text += "<br>Housing rights: " + a
    if not (row['employment_public_so'] and row['employment_public_gi'] and
            row['employment_private_so'] and row['employment_private_gi']):
        text += "<br>Employment rights:"
        if not (row['employment_public_so'] and row['employment_public_gi']):
            a = adder('employment_public_so', 'employment_public_gi', row)
            text += "<br>   public " + a
        if not (row['employment_private_so'] and row['employment_private_gi']):
            a = adder('employment_private_so','employment_private_gi', row)
            text += "<br>   private " + a
    return text

df_f['text'] = df.apply(make_text, axis=1)

## Create the plot

In [232]:
df_p = df_f.copy()
for col in df_p.columns:
    df_p[col] = df_p[col].astype(str)

data = [go.Choropleth(
    colorscale = 'Reds',
    locations = df_p['abbr'],
    z = df_p['opp_index'],
    locationmode = 'USA-states',
    text = df_p['text'],
    marker = go.choropleth.Marker(
        line = go.choropleth.marker.Line(
            color = 'rgb(255,255,255)',
            width = 1
        )),
    colorbar = go.choropleth.ColorBar(
        title = "opportunity index"),
)]

layout = go.Layout(
    title = go.layout.Title(text = 'Potential for states to pass LGBT anti-discrimination laws'),
    geo = go.layout.Geo(
        scope = 'usa',
        projection = go.layout.geo.Projection(type = 'albers usa'),
        showlakes = False)
)

fig = go.Figure(data = data, layout = layout)
py.iplot(fig, filename='anti-discrimination_lgbt')

# What about conversion therapy?

In [233]:
df_c = pd.DataFrame()
df_c['state'] = df['state']
df_c['abbr'] = df['abbr']
df_c['c'] = df['conversion_ban']
df_c['support'] = df['poll_favor']
df_c['control'] = round((df['dems_perc_senate'] + df['dems_perc_house'] + df['dem_gov'])/3,0)
df_c = df_c[df_c['c'] == 0]
df_c.drop('c', axis=1, inplace=True)
df_c['opp_index'] = round((norm(df_c['support']) + norm(df_c['control']))*100/2, 0)
df_c['text'] = df_c['state'] + "<br>Dem control: " + df_c['control'].astype(str)

In [234]:
df_p = df_c.copy()
for col in df_p.columns:
    df_p[col] = df_p[col].astype(str)

data = [go.Choropleth(
    colorscale = 'Blues',
    locations = df_p['abbr'],
    z = df_p['opp_index'],
    locationmode = 'USA-states',
    text = df_p['text'],
    marker = go.choropleth.Marker(
        line = go.choropleth.marker.Line(
            color = 'rgb(255,255,255)',
            width = 1
        )),
    colorbar = go.choropleth.ColorBar(
        title = "opportunity index"),
    reversescale=True
)]

layout = go.Layout(
    title = go.layout.Title(text = 'Potential for states ban conversion therapy for minors'),
    geo = go.layout.Geo(
        scope = 'usa',
        projection = go.layout.geo.Projection(type = 'albers usa'),
        showlakes = False)
)

fig = go.Figure(data = data, layout = layout)
py.iplot(fig, filename='conversion_lgbt')